In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)

Заменить файлы

In [3]:
stores = pd.read_csv('stores_vmi.csv', sep=';')
sales = pd.read_csv('06_2023_sales.csv', sep=';')
balance = pd.read_csv('06_2023_balance.csv', sep=';')

In [4]:
stores

,Склад,Крупность склада,ЦП,Город,Компания
0,АВТОМЕДОН-склад Готовой продукции,ЦС,Москва,Москва,Автомедон
1,"АВТОМЕДОН-склад Доп. ( ""Лешково"" )",ЦС,Москва,Москва,Автомедон
2,АВТОМЕДОН-склад Доп. ( 2 - й Этаж ) (АДП),ЦС,Москва,Москва,Автомедон
3,АВТОМЕДОН-склад Доп. ( 4 - й Этаж) (АДП),ЦС,Москва,Москва,Автомедон
4,АВТОМЕДОН-склад Доп. (Подвал) (Гольёвское),ЦС,Москва,Москва,Автомедон
...,...,...,...,...,...
120,Склад Партком (отв. хранение) Омск,"0,00",Новосибирск,Омск,Парт-Ком
121,Склад Партком (отв. хранение) Петрозаводск,"0,00",Санкт-Петербург,Петрозаводск,Парт-Ком
122,Склад Партком (отв. хранение) Тверь,"0,00",Москва,Тверь,Парт-Ком
123,Склад Партком (отв. хранение) Тольятти,"0,00",Самара,Тольятти,Парт-Ком


In [5]:
stores = stores.rename(columns={'Склад' : 'store_dir', 
                                'Крупность склада' : 'store_format_dir',
                                'ЦП' : 'cp_dir',
                                'Город' : 'city_dir',
                                'Компания' : 'company_dir'})

In [6]:
sales = sales.rename(columns={'Валовая прибыль' : 'detal', 
                              'Unnamed: 1' : 'abcxyz',
                              'Unnamed: 2' : 'brand',
                              'Unnamed: 3' : 'code1c',
                              'Unnamed: 4' : 'manufacturer',
                              'Unnamed: 5' : 'nomenclature', 
                              'Unnamed: 6' : 'quantity', 
                              'Unnamed: 7' : 'sales'})

In [7]:
sales = sales.iloc[11:]

In [8]:
sales = sales.dropna(how='all')

In [9]:
sales = sales.merge(stores, how='left', left_on='nomenclature', right_on='store_dir')

In [10]:
sales = sales.stack().str.replace(',','.').unstack()

In [11]:
sales['quantity'] = sales['quantity'].str.replace(' ','')
sales['sales'] = sales['sales'].str.replace(' ','')

In [12]:
sales['code1c'] = sales['code1c'].astype(float)
sales['quantity'] = sales['quantity'].astype(float)
sales['sales'] = sales['sales'].astype(float)

In [13]:
sales.dtypes

detal                object
abcxyz               object
brand                object
code1c              float64
manufacturer         object
nomenclature         object
quantity            float64
sales               float64
store_dir            object
store_format_dir     object
cp_dir               object
city_dir             object
company_dir          object
dtype: object

In [14]:
sales['store_dir'].ffill(inplace = True)
sales['store_format_dir'].ffill(inplace = True)
sales['cp_dir'].ffill(inplace=True)
sales['company_dir'].ffill(inplace=True)

In [15]:
sales = sales.dropna(how='all', subset=['abcxyz', 'brand', 'code1c', 'manufacturer', 'nomenclature'])

In [16]:
sales = sales.dropna(how='all', subset=['abcxyz', 'brand', 'code1c', 'manufacturer'])

In [17]:
sales

,detal,abcxyz,brand,code1c,manufacturer,nomenclature,quantity,sales,store_dir,store_format_dir,cp_dir,city_dir,company_dir
2,Амортизатор задний (2015),AY,AMD,4487.0,AMD.SA313,Амортизатор задний масляный 96316781/AMD.SA313...,10.0,9585.0,Красногорск-склад,ЦС,Москва,NaN,Автомедон
3,Амортизатор передний (2015),AX,AMD,4427.0,AMD.SA211,Амортизатор передний масляный 96187438/AMD.SA2...,20.0,20052.0,Красногорск-склад,ЦС,Москва,NaN,Автомедон
4,Барабан тормозной (2015),AX,AMD,51600.0,AMD.BD156,Барабан тормозной 96175281/AMD.BD156 AMD,40.0,45900.0,Красногорск-склад,ЦС,Москва,NaN,Автомедон
5,Барабан тормозной (2015),AX,AMD,51601.0,AMD.BD157,Барабан тормозной 96193771/AMD.BD157 AMD,100.0,169740.0,Красногорск-склад,ЦС,Москва,NaN,Автомедон
6,Бегунок трамблера (2015),BY,AMD,52563.0,AMD.I11,Бегунок трамблера 10495411/AMD.I11 AMD,10.0,1863.0,Красногорск-склад,ЦС,Москва,NaN,Автомедон
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42575,Шаровая опора (2015),AX,AMD,4537.0,AMD.BJ115,Шаровая опора 54530-25000/AMD.BJ115 AMD,1.0,316.0,Склад РОССКО (отв. хранение) Чебоксары,0.50,Нижний Новгород,NaN,РОССКО
42576,Шпилька (2015),BX,AMD,508505.0,AMD.HDKR55,Шпилька колеса заднего 52755-2G000/AMD.HDKR55 AMD,4.0,348.0,Склад РОССКО (отв. хранение) Чебоксары,0.50,Нижний Новгород,NaN,РОССКО
42577,Электробензонасос (мотор) (2015),AX,AMD,51125.0,AMD.FP143,Электробензонасос 31111-1R000/AMD.FP143 AMD,1.0,850.0,Склад РОССКО (отв. хранение) Чебоксары,0.50,Нижний Новгород,NaN,РОССКО
42578,Электробензонасос (мотор) (2015),AX,AMD,40323.0,AMD.FP127,Электробензонасос 31111-25000/AMD.FP127 AMD,2.0,1518.0,Склад РОССКО (отв. хранение) Чебоксары,0.50,Нижний Новгород,NaN,РОССКО


In [18]:
#тут надо организовать запись в файл

In [19]:
sales.groupby('store_format_dir').agg({'sales' : 'sum'})

,sales
store_format_dir,
0.00,1.461811e+05
0.15,1.366987e+06
0.50,6.555875e+06
1.00,9.155824e+06
5.00,2.103762e+07
ЦС,2.423950e+08


In [20]:
balance

,Стоимостная оценка склада в ценах номенклатуры,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,Период: на конец дня 30.06.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Показатели: Количество(в базовых ед.); Себесто...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Группировки строк: Склад (Элементы); Номенклат...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Отборы:\nСклад В группе из списка (Слады VMI; ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Дополнительные поля:\nСклад (Отдельно, Перед г...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
58003,Шланг тормозной (2015),AX,AMD,6770,AMD.BH38,Шланг тормозной передний 96212323/AMD.BH38 AMD,"1,000","104,60","174,00"
58004,Шпилька (2015),BX,AMD,508505,AMD.HDKR55,Шпилька колеса заднего 52755-2G000/AMD.HDKR55 AMD,"16,000","517,08","1 248,00"
58005,Электробензонасос (мотор) (2015),AX,AMD,510856,AMD.FP177,Электробензонасос 1602781/AMD.FP177 AMD,"2,000","895,00","1 436,00"
58006,Электробензонасос (мотор) (2015),AX,AMD,51125,AMD.FP143,Электробензонасос 31111-1R000/AMD.FP143 AMD,"2,000","1 049,03","1 378,00"


In [21]:
balance = balance.rename(columns={'Стоимостная оценка склада в ценах номенклатуры' : 'detal', 
                                  'Unnamed: 1' : 'abcxyz',
                                  'Unnamed: 2' : 'brand',
                                  'Unnamed: 3' : 'code1c',
                                  'Unnamed: 4' : 'manufacturer', 
                                  'Unnamed: 5' : 'nomenclature', 
                                  'Unnamed: 6' : 'quantity_os', 
                                  'Unnamed: 7' : 'sale_os',
                                  'Unnamed: 8' : 'sale_type_os'})

In [22]:
balance

,detal,abcxyz,brand,code1c,manufacturer,nomenclature,quantity_os,sale_os,sale_type_os
0,Период: на конец дня 30.06.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Показатели: Количество(в базовых ед.); Себесто...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Группировки строк: Склад (Элементы); Номенклат...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Отборы:\nСклад В группе из списка (Слады VMI; ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Дополнительные поля:\nСклад (Отдельно, Перед г...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
58003,Шланг тормозной (2015),AX,AMD,6770,AMD.BH38,Шланг тормозной передний 96212323/AMD.BH38 AMD,"1,000","104,60","174,00"
58004,Шпилька (2015),BX,AMD,508505,AMD.HDKR55,Шпилька колеса заднего 52755-2G000/AMD.HDKR55 AMD,"16,000","517,08","1 248,00"
58005,Электробензонасос (мотор) (2015),AX,AMD,510856,AMD.FP177,Электробензонасос 1602781/AMD.FP177 AMD,"2,000","895,00","1 436,00"
58006,Электробензонасос (мотор) (2015),AX,AMD,51125,AMD.FP143,Электробензонасос 31111-1R000/AMD.FP143 AMD,"2,000","1 049,03","1 378,00"


In [23]:
balance = balance.iloc[10:]

In [24]:
balance = balance.merge(stores, how='left', left_on='nomenclature', right_on='store_dir')

In [25]:
balance

,detal,abcxyz,brand,code1c,manufacturer,nomenclature,quantity_os,sale_os,sale_type_os,store_dir,store_format_dir,cp_dir,city_dir,company_dir
0,NaN,NaN,NaN,NaN,NaN,АВТОМЕДОН-склад Готовой продукции,"37 581,000","12 459,72",NaN,АВТОМЕДОН-склад Готовой продукции,ЦС,Москва,Москва,Автомедон
1,NaN,NaN,AMD,42686,85 мм. толщина для new принтер,Т/этикетки AMD 58*40 (цена 1 руб) (рулон 1000 ...,"36 881,000","12 312,83",NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,ASIN,501031,85 мм. толщина для new принтер,"Т/этикетки ASIN 58*40 ( цена 0,35 руб.) (рулон...","700,000","146,88",NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,"АВТОМЕДОН-склад Доп. ( ""Лешково"" )","2 350 565,000","556 597 994,23","718 655 376,00","АВТОМЕДОН-склад Доп. ( ""Лешково"" )",ЦС,Москва,Москва,Автомедон
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57993,Шланг тормозной (2015),AX,AMD,6770,AMD.BH38,Шланг тормозной передний 96212323/AMD.BH38 AMD,"1,000","104,60","174,00",NaN,NaN,NaN,NaN,NaN
57994,Шпилька (2015),BX,AMD,508505,AMD.HDKR55,Шпилька колеса заднего 52755-2G000/AMD.HDKR55 AMD,"16,000","517,08","1 248,00",NaN,NaN,NaN,NaN,NaN
57995,Электробензонасос (мотор) (2015),AX,AMD,510856,AMD.FP177,Электробензонасос 1602781/AMD.FP177 AMD,"2,000","895,00","1 436,00",NaN,NaN,NaN,NaN,NaN
57996,Электробензонасос (мотор) (2015),AX,AMD,51125,AMD.FP143,Электробензонасос 31111-1R000/AMD.FP143 AMD,"2,000","1 049,03","1 378,00",NaN,NaN,NaN,NaN,NaN


In [26]:
balance = balance.stack().str.replace(',','.').unstack()

In [27]:
balance

,detal,abcxyz,brand,code1c,manufacturer,nomenclature,quantity_os,sale_os,sale_type_os,store_dir,store_format_dir,cp_dir,city_dir,company_dir
0,NaN,NaN,NaN,NaN,NaN,АВТОМЕДОН-склад Готовой продукции,37 581.000,12 459.72,NaN,АВТОМЕДОН-склад Готовой продукции,ЦС,Москва,Москва,Автомедон
1,NaN,NaN,AMD,42686,85 мм. толщина для new принтер,Т/этикетки AMD 58*40 (цена 1 руб) (рулон 1000 ...,36 881.000,12 312.83,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,ASIN,501031,85 мм. толщина для new принтер,Т/этикетки ASIN 58*40 ( цена 0.35 руб.) (рулон...,700.000,146.88,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,"АВТОМЕДОН-склад Доп. ( ""Лешково"" )",2 350 565.000,556 597 994.23,718 655 376.00,"АВТОМЕДОН-склад Доп. ( ""Лешково"" )",ЦС,Москва,Москва,Автомедон
5,NaN,NaN,AMD,518309,AMD,"Пакет ""Майка""с лого ""AMD PARTS FROM KOREA""",960.000,2 848.80,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57993,Шланг тормозной (2015),AX,AMD,6770,AMD.BH38,Шланг тормозной передний 96212323/AMD.BH38 AMD,1.000,104.60,174.00,NaN,NaN,NaN,NaN,NaN
57994,Шпилька (2015),BX,AMD,508505,AMD.HDKR55,Шпилька колеса заднего 52755-2G000/AMD.HDKR55 AMD,16.000,517.08,1 248.00,NaN,NaN,NaN,NaN,NaN
57995,Электробензонасос (мотор) (2015),AX,AMD,510856,AMD.FP177,Электробензонасос 1602781/AMD.FP177 AMD,2.000,895.00,1 436.00,NaN,NaN,NaN,NaN,NaN
57996,Электробензонасос (мотор) (2015),AX,AMD,51125,AMD.FP143,Электробензонасос 31111-1R000/AMD.FP143 AMD,2.000,1 049.03,1 378.00,NaN,NaN,NaN,NaN,NaN


In [28]:
balance['quantity_os'] = balance['quantity_os'].str.replace(' ','')
balance['sale_os'] = balance['sale_os'].str.replace(' ','')
balance['sale_type_os'] = balance['sale_type_os'].str.replace(' ','')

In [29]:
balance

,detal,abcxyz,brand,code1c,manufacturer,nomenclature,quantity_os,sale_os,sale_type_os,store_dir,store_format_dir,cp_dir,city_dir,company_dir
0,NaN,NaN,NaN,NaN,NaN,АВТОМЕДОН-склад Готовой продукции,37581.000,12459.72,NaN,АВТОМЕДОН-склад Готовой продукции,ЦС,Москва,Москва,Автомедон
1,NaN,NaN,AMD,42686,85 мм. толщина для new принтер,Т/этикетки AMD 58*40 (цена 1 руб) (рулон 1000 ...,36881.000,12312.83,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,ASIN,501031,85 мм. толщина для new принтер,Т/этикетки ASIN 58*40 ( цена 0.35 руб.) (рулон...,700.000,146.88,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,"АВТОМЕДОН-склад Доп. ( ""Лешково"" )",2350565.000,556597994.23,718655376.00,"АВТОМЕДОН-склад Доп. ( ""Лешково"" )",ЦС,Москва,Москва,Автомедон
5,NaN,NaN,AMD,518309,AMD,"Пакет ""Майка""с лого ""AMD PARTS FROM KOREA""",960.000,2848.80,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57993,Шланг тормозной (2015),AX,AMD,6770,AMD.BH38,Шланг тормозной передний 96212323/AMD.BH38 AMD,1.000,104.60,174.00,NaN,NaN,NaN,NaN,NaN
57994,Шпилька (2015),BX,AMD,508505,AMD.HDKR55,Шпилька колеса заднего 52755-2G000/AMD.HDKR55 AMD,16.000,517.08,1248.00,NaN,NaN,NaN,NaN,NaN
57995,Электробензонасос (мотор) (2015),AX,AMD,510856,AMD.FP177,Электробензонасос 1602781/AMD.FP177 AMD,2.000,895.00,1436.00,NaN,NaN,NaN,NaN,NaN
57996,Электробензонасос (мотор) (2015),AX,AMD,51125,AMD.FP143,Электробензонасос 31111-1R000/AMD.FP143 AMD,2.000,1049.03,1378.00,NaN,NaN,NaN,NaN,NaN


In [30]:
balance['code1c'] = balance['code1c'].astype(float)
balance['quantity_os'] = balance['quantity_os'].astype(float)
balance['sale_os'] = balance['sale_os'].astype(float)
balance['sale_type_os'] = balance['sale_type_os'].astype(float)

In [31]:
balance

,detal,abcxyz,brand,code1c,manufacturer,nomenclature,quantity_os,sale_os,sale_type_os,store_dir,store_format_dir,cp_dir,city_dir,company_dir
0,NaN,NaN,NaN,NaN,NaN,АВТОМЕДОН-склад Готовой продукции,37581.0,1.245972e+04,NaN,АВТОМЕДОН-склад Готовой продукции,ЦС,Москва,Москва,Автомедон
1,NaN,NaN,AMD,42686.0,85 мм. толщина для new принтер,Т/этикетки AMD 58*40 (цена 1 руб) (рулон 1000 ...,36881.0,1.231283e+04,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,ASIN,501031.0,85 мм. толщина для new принтер,Т/этикетки ASIN 58*40 ( цена 0.35 руб.) (рулон...,700.0,1.468800e+02,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,"АВТОМЕДОН-склад Доп. ( ""Лешково"" )",2350565.0,5.565980e+08,718655376.0,"АВТОМЕДОН-склад Доп. ( ""Лешково"" )",ЦС,Москва,Москва,Автомедон
5,NaN,NaN,AMD,518309.0,AMD,"Пакет ""Майка""с лого ""AMD PARTS FROM KOREA""",960.0,2.848800e+03,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57993,Шланг тормозной (2015),AX,AMD,6770.0,AMD.BH38,Шланг тормозной передний 96212323/AMD.BH38 AMD,1.0,1.046000e+02,174.0,NaN,NaN,NaN,NaN,NaN
57994,Шпилька (2015),BX,AMD,508505.0,AMD.HDKR55,Шпилька колеса заднего 52755-2G000/AMD.HDKR55 AMD,16.0,5.170800e+02,1248.0,NaN,NaN,NaN,NaN,NaN
57995,Электробензонасос (мотор) (2015),AX,AMD,510856.0,AMD.FP177,Электробензонасос 1602781/AMD.FP177 AMD,2.0,8.950000e+02,1436.0,NaN,NaN,NaN,NaN,NaN
57996,Электробензонасос (мотор) (2015),AX,AMD,51125.0,AMD.FP143,Электробензонасос 31111-1R000/AMD.FP143 AMD,2.0,1.049030e+03,1378.0,NaN,NaN,NaN,NaN,NaN


In [32]:
balance.dtypes

detal                object
abcxyz               object
brand                object
code1c              float64
manufacturer         object
nomenclature         object
quantity_os         float64
sale_os             float64
sale_type_os        float64
store_dir            object
store_format_dir     object
cp_dir               object
city_dir             object
company_dir          object
dtype: object

In [33]:
balance['store_dir'].ffill(inplace = True)
balance['store_format_dir'].ffill(inplace = True)
balance['cp_dir'].ffill(inplace=True)
balance['company_dir'].ffill(inplace=True)

In [34]:
balance = balance.dropna(how='all', subset=['detal', 'abcxyz', 'brand', 'code1c', 'manufacturer'])

In [35]:
balance

,detal,abcxyz,brand,code1c,manufacturer,nomenclature,quantity_os,sale_os,sale_type_os,store_dir,store_format_dir,cp_dir,city_dir,company_dir
1,NaN,NaN,AMD,42686.0,85 мм. толщина для new принтер,Т/этикетки AMD 58*40 (цена 1 руб) (рулон 1000 ...,36881.0,12312.83,NaN,АВТОМЕДОН-склад Готовой продукции,ЦС,Москва,NaN,Автомедон
2,NaN,NaN,ASIN,501031.0,85 мм. толщина для new принтер,Т/этикетки ASIN 58*40 ( цена 0.35 руб.) (рулон...,700.0,146.88,NaN,АВТОМЕДОН-склад Готовой продукции,ЦС,Москва,NaN,Автомедон
5,NaN,NaN,AMD,518309.0,AMD,"Пакет ""Майка""с лого ""AMD PARTS FROM KOREA""",960.0,2848.80,NaN,"АВТОМЕДОН-склад Доп. ( ""Лешково"" )",ЦС,Москва,NaN,Автомедон
6,NaN,NaN,AMD,513978.0,45GWS,Перчатки с логотипом AMD (с сайтом),960.0,25440.00,NaN,"АВТОМЕДОН-склад Доп. ( ""Лешково"" )",ЦС,Москва,NaN,Автомедон
7,NaN,NaN,AMD,57136.0,41SF§,Упаковка универсальная /Комплект Фильтров/ (39...,960.0,23782.17,NaN,"АВТОМЕДОН-склад Доп. ( ""Лешково"" )",ЦС,Москва,NaN,Автомедон
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57993,Шланг тормозной (2015),AX,AMD,6770.0,AMD.BH38,Шланг тормозной передний 96212323/AMD.BH38 AMD,1.0,104.60,174.0,Склад РОССКО (отв. хранение) Чебоксары,0.50,Нижний Новгород,NaN,РОССКО
57994,Шпилька (2015),BX,AMD,508505.0,AMD.HDKR55,Шпилька колеса заднего 52755-2G000/AMD.HDKR55 AMD,16.0,517.08,1248.0,Склад РОССКО (отв. хранение) Чебоксары,0.50,Нижний Новгород,NaN,РОССКО
57995,Электробензонасос (мотор) (2015),AX,AMD,510856.0,AMD.FP177,Электробензонасос 1602781/AMD.FP177 AMD,2.0,895.00,1436.0,Склад РОССКО (отв. хранение) Чебоксары,0.50,Нижний Новгород,NaN,РОССКО
57996,Электробензонасос (мотор) (2015),AX,AMD,51125.0,AMD.FP143,Электробензонасос 31111-1R000/AMD.FP143 AMD,2.0,1049.03,1378.0,Склад РОССКО (отв. хранение) Чебоксары,0.50,Нижний Новгород,NaN,РОССКО


In [36]:
sales['code1c'] = sales['code1c'].astype(int)

In [37]:
balance['code1c'] = balance['code1c'].astype(int)

C:\Users\alekseeva_ka\AppData\Local\Temp\ipykernel_5876\1297243457.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balance['code1c'] = balance['code1c'].astype(int)


In [38]:
sales.dtypes

detal                object
abcxyz               object
brand                object
code1c                int32
manufacturer         object
nomenclature         object
quantity            float64
sales               float64
store_dir            object
store_format_dir     object
cp_dir               object
city_dir             object
company_dir          object
dtype: object

In [39]:
total = pd.concat([sales, balance])

In [40]:
total

,detal,abcxyz,brand,code1c,manufacturer,nomenclature,quantity,sales,store_dir,store_format_dir,cp_dir,city_dir,company_dir,quantity_os,sale_os,sale_type_os
2,Амортизатор задний (2015),AY,AMD,4487,AMD.SA313,Амортизатор задний масляный 96316781/AMD.SA313...,10.0,9585.0,Красногорск-склад,ЦС,Москва,NaN,Автомедон,NaN,NaN,NaN
3,Амортизатор передний (2015),AX,AMD,4427,AMD.SA211,Амортизатор передний масляный 96187438/AMD.SA2...,20.0,20052.0,Красногорск-склад,ЦС,Москва,NaN,Автомедон,NaN,NaN,NaN
4,Барабан тормозной (2015),AX,AMD,51600,AMD.BD156,Барабан тормозной 96175281/AMD.BD156 AMD,40.0,45900.0,Красногорск-склад,ЦС,Москва,NaN,Автомедон,NaN,NaN,NaN
5,Барабан тормозной (2015),AX,AMD,51601,AMD.BD157,Барабан тормозной 96193771/AMD.BD157 AMD,100.0,169740.0,Красногорск-склад,ЦС,Москва,NaN,Автомедон,NaN,NaN,NaN
6,Бегунок трамблера (2015),BY,AMD,52563,AMD.I11,Бегунок трамблера 10495411/AMD.I11 AMD,10.0,1863.0,Красногорск-склад,ЦС,Москва,NaN,Автомедон,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57993,Шланг тормозной (2015),AX,AMD,6770,AMD.BH38,Шланг тормозной передний 96212323/AMD.BH38 AMD,NaN,NaN,Склад РОССКО (отв. хранение) Чебоксары,0.50,Нижний Новгород,NaN,РОССКО,1.0,104.60,174.0
57994,Шпилька (2015),BX,AMD,508505,AMD.HDKR55,Шпилька колеса заднего 52755-2G000/AMD.HDKR55 AMD,NaN,NaN,Склад РОССКО (отв. хранение) Чебоксары,0.50,Нижний Новгород,NaN,РОССКО,16.0,517.08,1248.0
57995,Электробензонасос (мотор) (2015),AX,AMD,510856,AMD.FP177,Электробензонасос 1602781/AMD.FP177 AMD,NaN,NaN,Склад РОССКО (отв. хранение) Чебоксары,0.50,Нижний Новгород,NaN,РОССКО,2.0,895.00,1436.0
57996,Электробензонасос (мотор) (2015),AX,AMD,51125,AMD.FP143,Электробензонасос 31111-1R000/AMD.FP143 AMD,NaN,NaN,Склад РОССКО (отв. хранение) Чебоксары,0.50,Нижний Новгород,NaN,РОССКО,2.0,1049.03,1378.0


In [41]:
total.dtypes

detal                object
abcxyz               object
brand                object
code1c                int32
manufacturer         object
nomenclature         object
quantity            float64
sales               float64
store_dir            object
store_format_dir     object
cp_dir               object
city_dir             object
company_dir          object
quantity_os         float64
sale_os             float64
sale_type_os        float64
dtype: object

ВПИСАТЬ ДАТУ

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [42]:
total['data'] = '01.06.2023'

In [43]:
total = total.rename(columns={'detal' : 'Деталь',
                              'abcxyz' : 'ABC\XYZ', 
                              'brand' : 'Бренд',
                              'code1c' : 'Код 1С',
                              'manufacturer' : 'Производитель',
                              'nomenclature' : 'Номенклатура',
                              'quantity' : 'Продано, шт.',
                              'sales' : 'Продано, руб.', 
                              'store_dir' : 'Склад',
                              'store_format_dir' : 'Крупность склада',
                              'cp_dir' : 'Центр продаж',
                              'city_dir' : 'Город',
                              'company_dir' : 'Компания',
                              'quantity_os' : 'Запас, шт.',
                              'sale_os' : 'Себестоимость, руб.',
                              'sale_type_os' : 'Запас, руб.', 
                              'data' : 'Дата'
                             })

In [44]:
total

,Деталь,ABC\XYZ,Бренд,Код 1С,Производитель,Номенклатура,"Продано, шт.","Продано, руб.",Склад,Крупность склада,Центр продаж,Город,Компания,"Запас, шт.","Себестоимость, руб.","Запас, руб.",Дата
2,Амортизатор задний (2015),AY,AMD,4487,AMD.SA313,Амортизатор задний масляный 96316781/AMD.SA313...,10.0,9585.0,Красногорск-склад,ЦС,Москва,NaN,Автомедон,NaN,NaN,NaN,01.06.2023
3,Амортизатор передний (2015),AX,AMD,4427,AMD.SA211,Амортизатор передний масляный 96187438/AMD.SA2...,20.0,20052.0,Красногорск-склад,ЦС,Москва,NaN,Автомедон,NaN,NaN,NaN,01.06.2023
4,Барабан тормозной (2015),AX,AMD,51600,AMD.BD156,Барабан тормозной 96175281/AMD.BD156 AMD,40.0,45900.0,Красногорск-склад,ЦС,Москва,NaN,Автомедон,NaN,NaN,NaN,01.06.2023
5,Барабан тормозной (2015),AX,AMD,51601,AMD.BD157,Барабан тормозной 96193771/AMD.BD157 AMD,100.0,169740.0,Красногорск-склад,ЦС,Москва,NaN,Автомедон,NaN,NaN,NaN,01.06.2023
6,Бегунок трамблера (2015),BY,AMD,52563,AMD.I11,Бегунок трамблера 10495411/AMD.I11 AMD,10.0,1863.0,Красногорск-склад,ЦС,Москва,NaN,Автомедон,NaN,NaN,NaN,01.06.2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57993,Шланг тормозной (2015),AX,AMD,6770,AMD.BH38,Шланг тормозной передний 96212323/AMD.BH38 AMD,NaN,NaN,Склад РОССКО (отв. хранение) Чебоксары,0.50,Нижний Новгород,NaN,РОССКО,1.0,104.60,174.0,01.06.2023
57994,Шпилька (2015),BX,AMD,508505,AMD.HDKR55,Шпилька колеса заднего 52755-2G000/AMD.HDKR55 AMD,NaN,NaN,Склад РОССКО (отв. хранение) Чебоксары,0.50,Нижний Новгород,NaN,РОССКО,16.0,517.08,1248.0,01.06.2023
57995,Электробензонасос (мотор) (2015),AX,AMD,510856,AMD.FP177,Электробензонасос 1602781/AMD.FP177 AMD,NaN,NaN,Склад РОССКО (отв. хранение) Чебоксары,0.50,Нижний Новгород,NaN,РОССКО,2.0,895.00,1436.0,01.06.2023
57996,Электробензонасос (мотор) (2015),AX,AMD,51125,AMD.FP143,Электробензонасос 31111-1R000/AMD.FP143 AMD,NaN,NaN,Склад РОССКО (отв. хранение) Чебоксары,0.50,Нижний Новгород,NaN,РОССКО,2.0,1049.03,1378.0,01.06.2023


ВПИСАТЬ ДАТУ

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [45]:
total.to_csv('KPI_06_2023.csv', sep=';', decimal=',', encoding='ANSI')

In [46]:
stores

,store_dir,store_format_dir,cp_dir,city_dir,company_dir
0,АВТОМЕДОН-склад Готовой продукции,ЦС,Москва,Москва,Автомедон
1,"АВТОМЕДОН-склад Доп. ( ""Лешково"" )",ЦС,Москва,Москва,Автомедон
2,АВТОМЕДОН-склад Доп. ( 2 - й Этаж ) (АДП),ЦС,Москва,Москва,Автомедон
3,АВТОМЕДОН-склад Доп. ( 4 - й Этаж) (АДП),ЦС,Москва,Москва,Автомедон
4,АВТОМЕДОН-склад Доп. (Подвал) (Гольёвское),ЦС,Москва,Москва,Автомедон
...,...,...,...,...,...
120,Склад Партком (отв. хранение) Омск,"0,00",Новосибирск,Омск,Парт-Ком
121,Склад Партком (отв. хранение) Петрозаводск,"0,00",Санкт-Петербург,Петрозаводск,Парт-Ком
122,Склад Партком (отв. хранение) Тверь,"0,00",Москва,Тверь,Парт-Ком
123,Склад Партком (отв. хранение) Тольятти,"0,00",Самара,Тольятти,Парт-Ком
